# **Initializers**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

In [ ]:
%cd /content/gdrive/My Drive/08_emotion/
base_dir='/content/gdrive/My Drive/08_emotion/'

In [ ]:
import os
import numpy as np

import tensorflow as tf
'''from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.nasnet import NASNetMobile
from tensorflow.keras.applications.inception_v3 import InceptionV3'''
from keras.applications import vgg16
from keras.applications import xception
from keras.applications import densenet
from keras.applications import vgg19

from tensorflow.keras import optimizers
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Layer,Dense, Dropout, Input, Activation, TimeDistributed, Reshape
from tensorflow.keras.layers import  GRU, Bidirectional
from tensorflow.keras.layers import  Conv1D, Conv2D, MaxPooling2D, Flatten, BatchNormalization, LSTM, ZeroPadding2D, GlobalAveragePooling2D, SpatialDropout2D, GlobalMaxPool1D,Convolution1D
from tensorflow.keras.callbacks import History 
from tensorflow.keras.models import Model
from keras.utils.vis_utils import plot_model
from keras.utils import np_utils

import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import confusion_matrix
import time
import math
import random
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

def smooth_curve(points, factor=0.0):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

def plot_results(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(1, len(acc) + 1)
  %matplotlib inline
  plt.plot(epochs, smooth_curve(acc), 'bo', label='Training acc')
  plt.plot(epochs, smooth_curve(val_acc), 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()
  plt.figure()
  plt.plot(epochs, smooth_curve(loss), 'bo', label='Training loss')
  plt.plot(epochs, smooth_curve(val_loss), 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()
  plt.show()

def calculate_test_accuracy():
  fold_results = {}
  Y_test = np_utils.to_categorical(Y_test)
  predicted_probas = model.predict(np.array(X_test),steps = len(X_test))
  fold_results['predicted_probas'] = predicted_probas
  binary_prediction=[]
  binary_prediction0=[]
  for i in range(len(predicted_probas)):
     binary_prediction0 = 1 if predicted_probas[i]>=0.5 else 0
     # binary_prediction0 = np.round(predicted_probas)
     binary_prediction.append(binary_prediction0)
  fold_results['binary_prediction'] = binary_prediction
  conf_mat = confusion_matrix(binary_prediction, Y_test)
  fold_results['confusion_matrix'] = conf_mat
  acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
  fold_results['test_accuracy'] = acc
  print('Overall accuracy: {:.2f} %'.format(acc*100))
  return fold_results

# **Load Connectivity images**

In [ ]:
from natsort import natsorted
import numpy as np
import os
base_dir1 = '/content/gdrive/My Drive/08_emotion/data/emotion-fused images/q1/'
q1_data = os.listdir(base_dir1)
q1_data = natsorted(q1_data)

base_dir2 = '/content/gdrive/My Drive/08_emotion/data/emotion-fused images/q2/'
q2_data = os.listdir(base_dir2)
q2_data = natsorted(q2_data)

base_dir3 = '/content/gdrive/My Drive/08_emotion/data/emotion-fused images/q3/'
q3_data = os.listdir(base_dir3)
q3_data = natsorted(q3_data)

base_dir4 = '/content/gdrive/My Drive/08_emotion/data/emotion-fused images/q4/'
q4_data = os.listdir(base_dir4)
q4_data = natsorted(q4_data)

In [ ]:
unwanted_num = {'fi(4626)q2 (1).jpg', 'fi(4627)q2 (1).jpg', 'fi(4628)q2 (1).jpg', 'fi(4629)q2 (1).jpg', 'fi(4630)q2 (1).jpg', 'fi(4631)q2 (1).jpg', 'fi(4632)q2 (1).jpg'}
list1 = [ele for ele in q2_data if ele not in unwanted_num]
q2_data=list1

In [ ]:
import cv2
import pickle
base_dir = '/content/gdrive/My Drive/08_emotion/'

X_q1 = []
for img in q1_data:
    image = cv2.imread(base_dir1+img)
    image = cv2.resize(image,(224,224))
    X_q1.append(image)
with open(base_dir+'q1', 'wb') as file_pi1:
    pickle.dump(X_q1, file_pi1)
    
X_q2 = []
for img in q2_data:
    image = cv2.imread(base_dir2+img)
    image = cv2.resize(image,(224,224))
    X_q2.append(image)
with open(base_dir+'q2', 'wb') as file_pi1:
    pickle.dump(X_q2, file_pi1)

X_q3 = []
for img in q3_data:
    image = cv2.imread(base_dir3+img)
    image = cv2.resize(image,(224,224))
    X_q3.append(image)
with open(base_dir+'q3', 'wb') as file_pi1:
    pickle.dump(X_q3, file_pi1)

X_q4 = []
for img in q4_data:
    image = cv2.imread(base_dir4+img)
    image = cv2.resize(image,(224,224))
    X_q4.append(image)
with open(base_dir+'q4', 'wb') as file_pi1:
    pickle.dump(X_q4, file_pi1)

###load Images from Drive

In [ ]:
with open(base_dir+'q1', 'rb') as f:
    X_q1 = pickle.load(f)

with open(base_dir+'q2', 'rb') as f:
    X_q2 = pickle.load(f)

with open(base_dir+'q3', 'rb') as f:
    X_q3 = pickle.load(f)

with open(base_dir+'q4', 'rb') as f:
    X_q4 = pickle.load(f)

In [ ]:
Images_data = X_q1+X_q2+X_q3+X_q4
import pickle
base_dir = '/content/gdrive/My Drive/08_emotion/'
with open(base_dir+'Images_40clips_32subjects', 'wb') as file_pi1:
    pickle.dump(Images_data, file_pi1)

Load from drive

In [ ]:
with open(base_dir+'Images_40clips_32subjects', 'rb') as f:
    Images_data = pickle.load(f)

In [ ]:
q1 = 5760*[0]
q2 = 5760*[1]
q3 = 5760*[2]
q4 = 5760*[3]

labels = q1+q2+q3+q4
#labels

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(Images_data,labels,test_size=0.1, random_state=6)

# **TL-EfficientNetb0**

In [ ]:

def create_model():
  conv_base = tf.keras.applications.EfficientNetB0(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
  conv_base.trainable = True
  '''set_trainable = True
  for layer in conv_base.layers:
      if layer.name == 'block5_conv3':
          set_trainable = True
      if set_trainable:
          layer.trainable = True
      else:
          layer.trainable = False
  print("Model edited")'''

  model = models.Sequential()
  model.add(conv_base)
  model.add(layers.Flatten())
  #model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(128)) #,kernel_regularizer=regularizers.l1_l2(l1=1e-6, l2=1e-5),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
  model.add(BatchNormalization())
  model.add(layers.Activation('relu'))
  model.add(layers.Dropout(0.3))
  #model.add(layers.Dense(1, activation='sigmoid'))
  model.add(layers.Dense(1))
  model.add(BatchNormalization())
  model.add(layers.Activation('sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(learning_rate=3e-5),metrics=['acc'])
  return model


fold_count=0
hist_VGG16 = {}
seed = 7
np.random.seed(seed)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

for train_index, val_index in skf.split(X_train,Y_train):
    fold_count = fold_count + 1
    #if fold_count <7:
    #  continue;
    x_train, x_val = np.array(X_train)[train_index], np.array(X_train)[val_index]
    y_train, y_val = np.array(Y_train)[train_index], np.array(Y_train)[val_index]

    model = create_model()
    history = model.fit(x_train,y_train,epochs=50, batch_size=16, validation_data = (x_val,y_val), shuffle=True)
    plot_results(history)
    
    hist_VGG16['fold'+str(fold_count)+'_history']=history.history
    hist_VGG16['fold'+str(fold_count)+'_results'] = calculate_test_accuracy()
    print('fold_'+str(fold_count)+' done!')
    with open(base_dir+'results/hist_VGG16_fold_'+str(fold_count), 'wb') as file_pi1:
       pickle.dump(hist_VGG16, file_pi1)

58900480/58889256 [==============================] - 0s 0us/step
Epoch 1/50


# **Prepare the video Samples**

In [ ]:
# prepare video samples from scatch or from previously loaded Images_data

In [ ]:
# Alternatively use previously saved Images_data
import pickle
import numpy as np
base_dir = '/content/gdrive/My Drive/08_emotion/'
Images_data = pickle.load(open(base_dir+'Images_40clips_32subjects', "rb" ))

X_q1 = Images_data[:5760]
X_q2 = Images_data[5760:2*5760]
X_q3 = Images_data[2*5760:3*5760]
X_q4 = Images_data[3*5760:4*5760]

In [ ]:
Images_data = []

In [ ]:
video_q1 = []
subj = 0
stride = 1
init_frame = 0
frame_length = 4
for subj in range(32*40): # number of subjects
  for img in range(4): #stride 1/number of images of each subject
       if img+frame_length>4:
          break
       tmp_video = X_q1[subj*4+img:subj*4+img+frame_length]
       video_q1.append(tmp_video)
print(np.shape(video_q1))
print(img)

(1280, 4, 224, 224, 3)
1


In [ ]:
video_q2 = []
subj = 0
stride = 1
init_frame = 0
frame_length = 4
for subj in range(32*40): # number of subjects
  for img in range(4): #stride 1/number of images of each subject
       if img+frame_length>4:
          break
       tmp_video = X_q2[subj*4+img:subj*4+img+frame_length]
       video_q2.append(tmp_video)
print(np.shape(video_q2))
print(img)

(1280, 4, 224, 224, 3)
1


In [ ]:
video_q3 = []
subj = 0
stride = 1
init_frame = 0
frame_length = 4
for subj in range(32*40): # number of subjects
  for img in range(4): #stride 1/number of images of each subject
       if img+frame_length>4:
          break
       tmp_video = X_q3[subj*4+img:subj*4+img+frame_length]
       video_q3.append(tmp_video)
print(np.shape(video_q3))
print(img)

(1280, 4, 224, 224, 3)
1


In [ ]:
video_q4 = []
subj = 0
stride = 1
init_frame = 0
frame_length = 4
for subj in range(32*40): # number of subjects
  for img in range(4): #stride 1/number of images of each subject
       if img+frame_length>4:
          break
       tmp_video = X_q4[subj*4+img:subj*4+img+frame_length]
       video_q4.append(tmp_video)
print(np.shape(video_q4))
print(img)

(1280, 4, 224, 224, 3)
1


In [ ]:
video_data=video_q1+video_q2+video_q3+video_q4
video_q1,video_q2,video_q3,video_q4,X_q1,X_q2,X_q3,X_q4 = [],[],[],[],[],[],[],[]
#del video_sz

In [ ]:
import pickle 
base_dir = '/content/gdrive/My Drive/08_emotion/'
with open(base_dir+'video_data', 'wb') as file_pi1:
    pickle.dump(video_data, file_pi1)

### **load video from drive**

In [ ]:
# Alternatively use previously saved video_data
import pickle
import numpy as np

base_dir = '/content/gdrive/My Drive/08_emotion/'
video_data = pickle.load(open(base_dir+'video_data', "rb" ))

In [ ]:
q1 = 1280*[0]
q2 = 1280*[1]
q3 = 1280*[2]
q4 = 1280*[3]

labels = q1+q2+q3+q4
#labels

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(video_data,labels,test_size=0.1, random_state=8)

In [ ]:
video_data=[]

**bold text**# **EfficientnetB0-LSTM**

In [ ]:
def create_model():
  conv_base = tf.keras.applications.EfficientNetB0(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
  conv_base.trainable = True
  '''set_trainable = False
  for layer in conv_base.layers:
      if layer.name == 'block5_conv2':
          set_trainable = True
      if set_trainable:
          layer.trainable = True
      else:
          layer.trainable = False
  print("Model edited")'''

  model = models.Sequential()
  model.add(conv_base)
  model.add(layers.Flatten())
  model.add(layers.Dense(32)) #,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
  
  '''model.add(BatchNormalization())
  model.add(layers.Activation('sigmoid'))'''
  cnn = models.Model(model.input, model.output)

  seq_input = Input(shape=(4,224,224,3))
  encoded_sequence = TimeDistributed(cnn)(seq_input)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=True))(encoded_sequence)
  
  #encoded_sequence = Dropout(rate=0.1)(encoded_sequence)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=False))(encoded_sequence)
  out = Dense(4, activation="softmax")(encoded_sequence)
  #out = Convolution1D(1, kernel_size=1, activation="sigmoid", padding="same")(encoded_sequence)

  cnn_lstm = models.Model(seq_input, out)
  model.summary()
  
  cnn_lstm.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=1e-4),metrics=['acc'])
  if fold_count == 1:
    cnn_lstm.summary()
  return cnn_lstm

fold_count=0
hist_EfficientNetB0LSTM = {}
seed = 7
np.random.seed(seed)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

for train_index, val_index in skf.split(X_train,Y_train):
    fold_count = fold_count + 1
    #if fold_count < 8:
    #  continue;
    x_train, x_val = np.array(X_train)[train_index], np.array(X_train)[val_index]
    y_train, y_val = np.array(Y_train)[train_index], np.array(Y_train)[val_index]
    y_train = np_utils.to_categorical(y_train)
    y_val = np_utils.to_categorical(y_val)

    model = create_model()
    from keras.utils.vis_utils import plot_model
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
    history = model.fit(x_train,y_train,epochs=50, batch_size=10, validation_data = (x_val,y_val), shuffle=True)
    plot_results(history)
    
    hist_EfficientNetB0LSTM['fold'+str(fold_count)+'_history']= history.history
    hist_EfficientNetB0LSTM['fold'+str(fold_count)+'_results'] = calculate_test_accuracy()
    print('fold_'+str(fold_count)+' done!')
    with open(base_dir+'hist_EfficientNetB0LSTM_fold_'+str(fold_count), 'wb') as file_pi1:
        pickle.dump(hist_EfficientNetB0LSTM, file_pi1)

In [ ]:
model = models.Sequential()
  model.add(conv_base)
  model.add(layers.Flatten())
  model.add(layers.Dense(32)) #,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
  
  '''model.add(BatchNormalization())
  model.add(layers.Activation('sigmoid'))'''
  cnn = models.Model(model.input, model.output)

  seq_input = Input(shape=(4,224,224,3))
  encoded_sequence = TimeDistributed(cnn)(seq_input)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=True))(encoded_sequence)
  
  #encoded_sequence = Dropout(rate=0.1)(encoded_sequence)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=False))(encoded_sequence)
  out = Dense(4, activation="softmax")(encoded_sequence)
  #out = Convolution1D(1, kernel_size=1, activation="sigmoid", padding="same")(encoded_sequence)

  cnn_lstm = models.Model(seq_input, out)
  model.summary()

In [ ]:
fold_results = {}
Y_test = np_utils.to_categorical(Y_test)
predicted_probas = model.predict(np.array(X_test),steps = len(X_test))
fold_results['predicted_probas'] = predicted_probas
maxindex = predicted_probas[i,:].argmax()
binary_prediction=[]
binary_prediction0=[]
for i in range(len(predicted_probas)):
    binary_prediction0 = 1 if predicted_probas[i,:].argmax()>=0.5 else 0
    # binary_prediction0 = np.round(predicted_probas)
    binary_prediction.append(binary_prediction0)
fold_results['binary_prediction'] = binary_prediction
conf_mat = confusion_matrix(binary_prediction, Y_test)
fold_results['confusion_matrix'] = conf_mat
acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
fold_results['test_accuracy'] = acc
print('Overall accuracy: {:.2f} %'.format(acc*100))

In [ ]:
predicted_probas

In [ ]:
def create_model():
  conv_base = tf.keras.applications.EfficientNetB0(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
  conv_base.trainable = True
  '''set_trainable = False
  for layer in conv_base.layers:
      if layer.name == 'block5_conv2':
          set_trainable = True
      if set_trainable:
          layer.trainable = True
      else:
          layer.trainable = False
  print("Model edited")'''

  model = models.Sequential()
  model.add(conv_base)
  model.add(layers.Flatten())
  model.add(layers.Dense(32)) #,kernel_regularizer=regularizers.l1_l2(l1=1e-5, l2=1e-4),bias_regularizer=regularizers.l2(1e-4),activity_regularizer=regularizers.l2(1e-5)))
  
  '''model.add(BatchNormalization())
  model.add(layers.Activation('sigmoid'))'''
  cnn = models.Model(model.input, model.output)

  seq_input = Input(shape=(4,224,224,3))
  encoded_sequence = TimeDistributed(cnn)(seq_input)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=True))(encoded_sequence)
  
  #encoded_sequence = Dropout(rate=0.1)(encoded_sequence)
  encoded_sequence = Bidirectional(LSTM(32, return_sequences=False))(encoded_sequence)
  out = Dense(4, activation="softmax")(encoded_sequence)
  #out = Convolution1D(1, kernel_size=1, activation="sigmoid", padding="same")(encoded_sequence)

  cnn_lstm = models.Model(seq_input, out)
  cnn_lstm.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=1e-4),metrics=['acc'])
  if fold_count == 1:
    cnn_lstm.summary()
  return cnn_lstm

fold_count=0
hist_EfficientNetB0LSTM = {}
seed = 7
np.random.seed(seed)
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

for train_index, val_index in skf.split(X_train,Y_train):
    fold_count = fold_count + 1
    #if fold_count < 8:
    #  continue;
    x_train, x_val = np.array(X_train)[train_index], np.array(X_train)[val_index]
    y_train, y_val = np.array(Y_train)[train_index], np.array(Y_train)[val_index]
    y_train = np_utils.to_categorical(y_train)
    y_val = np_utils.to_categorical(y_val)

    model = create_model()
    from keras.utils.vis_utils import plot_model
    #plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)
    history = model.fit(x_train,y_train,epochs=50, batch_size=10, validation_data = (x_val,y_val), shuffle=True)
    plot_results(history)
    
    hist_EfficientNetB0LSTM['fold'+str(fold_count)+'_history']= history.history
    hist_EfficientNetB0LSTM['fold'+str(fold_count)+'_results'] = calculate_test_accuracy()
    print('fold_'+str(fold_count)+' done!')
    with open(base_dir+'hist_EfficientNetB0LSTM_fold_'+str(fold_count), 'wb') as file_pi1:
        pickle.dump(hist_EfficientNetB0LSTM, file_pi1)

16719872/16705208 [==============================] - 0s 0us/step
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 4, 224, 224, 3)]  0         
                                                                 
 time_distributed (TimeDistr  (None, 4, 32)            6056643   
 ibuted)                                                         
                                                                 
 bidirectional (Bidirectiona  (None, 4, 64)            16640     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               24832     
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 4)                 260  